# Earth System Science Data 크롤러

- 2009 ~ 2019년 () 게재 논문 저자와 이메일을 크롤링하는 크롤러입니다.
- 크롤링 결과는 'Platform', 'Year', 'Volume', 'Author', 'Email', 'URL'의 6개 칼럼으로 구성된 데이터프레임으로 정리하였습니다.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [ ]:
'https://www.earth-syst-sci-data.net/volumes.html'

In [ ]:
df = pd.DataFrame(columns = ['Platform', 'Year', 'Volume', 'Author', 'Email', 'URL'])

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

for i in range(1, 28):
    url = f"https://www.sciencedirect.com/journal/data-in-brief/vol/{i}/suppl/C"
    driver.get(url)
    elem = driver.find_element_by_tag_name("body")
    elem.send_keys(Keys.PAGE_DOWN)
    articles = driver.find_elements_by_class_name('anchor.article-content-title.u-margin-xs-top.u-margin-s-bottom')
    article_url = [article.get_attribute('href') for article in articles]
    for k in range(len(article_url)):
        driver.get(article_url[k])
        view_more = driver.find_elements_by_class_name('author.size-m.workspace-trigger')
        if len(view_more) != 0:
            for item in view_more:
                envelope = item.find_elements_by_class_name('icon.icon-envelope')
                if not not envelope:
                    item.click()
#                     wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'WorkspaceAuthor.text-s')))
#                     wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'e-address')))
                    author = driver.find_element_by_class_name('WorkspaceAuthor.text-s').find_element_by_class_name('author').text
                    email = driver.find_element_by_class_name('e-address').text
                    year = driver.find_element_by_class_name('text-xs').text.split()[3].replace(',', '')
                    df = df.append({'Platform': 'Data in Brief', 'Year': year, 'Volume': i, 'Author': author, 'Email': email, 'URL': article_url[k] }, ignore_index=True)
#                     wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'icon.icon-cross')))
                    elem = driver.find_element_by_tag_name("body")
                    elem.send_keys(Keys.RETURN)
print('finished')